In [5]:
import sys
sys.path.append('/Users/tiansu/Documents/git/ml4t/ML4T_2017Summer')
import pandas as pd
import numpy as np
import datetime as dt
from util import get_data, plot_data

In [217]:
orders_file = 'orders/orders-12-modified.csv'
start_val = 1000000
orders = pd.read_csv(orders_file, index_col=0, parse_dates=True, sep=',')
orders = orders.sort_index()
# orders = orders[:4]
orders

,Symbol,Order,Shares
Date,,,
2011-01-10,XOM,BUY,1500
2011-01-13,XOM,SELL,1500
2011-01-13,IBM,BUY,4000
2011-01-26,GOOG,BUY,1000
2011-02-02,AAPL,SELL,400
2011-02-10,AAPL,BUY,400
2011-03-03,GOOG,SELL,1000
2011-03-03,IBM,SELL,2200
2011-05-03,IBM,BUY,1500


In [218]:
dates = pd.date_range(orders.index[0], orders.index[-1])
symbols = orders['Symbol'].unique().tolist()

In [219]:
prices = get_data(symbols, dates)[symbols] 
prices = pd.concat([prices, pd.DataFrame(index=dates)],axis=1) 
prices = prices.fillna(method='ffill')
prices

,XOM,IBM,GOOG,AAPL
2011-01-10,72.02,143.41,614.21,340.99
2011-01-11,72.56,143.06,616.01,340.18
2011-01-12,73.41,144.82,616.87,342.95
2011-01-13,73.54,144.55,616.69,344.20
2011-01-14,74.62,145.70,624.18,346.99
2011-01-15,74.62,145.70,624.18,346.99
2011-01-16,74.62,145.70,624.18,346.99
2011-01-17,74.62,145.70,624.18,346.99
2011-01-18,75.45,146.33,639.63,339.19
2011-01-19,75.00,151.22,631.75,337.39


In [220]:
col = ['order', 'leverage', 'transaction'] + symbols
trade = pd.DataFrame(columns=col+['cash'], index=[dates])
trade.ix[0, ['order']] = start_val
trade[col] = trade[col].fillna(value=0)
trade.ix[0, 'cash'] = 0
trade = trade.merge(prices, how='left', left_index=True, right_index=True, suffixes=('', '_price'))
trade

,order,leverage,transaction,XOM,IBM,GOOG,AAPL,cash,XOM_price,IBM_price,GOOG_price,AAPL_price
2011-01-10,1000000,0,0,0,0,0,0,0,72.02,143.41,614.21,340.99
2011-01-11,0,0,0,0,0,0,0,NaN,72.56,143.06,616.01,340.18
2011-01-12,0,0,0,0,0,0,0,NaN,73.41,144.82,616.87,342.95
2011-01-13,0,0,0,0,0,0,0,NaN,73.54,144.55,616.69,344.20
2011-01-14,0,0,0,0,0,0,0,NaN,74.62,145.70,624.18,346.99
2011-01-15,0,0,0,0,0,0,0,NaN,74.62,145.70,624.18,346.99
2011-01-16,0,0,0,0,0,0,0,NaN,74.62,145.70,624.18,346.99
2011-01-17,0,0,0,0,0,0,0,NaN,74.62,145.70,624.18,346.99
2011-01-18,0,0,0,0,0,0,0,NaN,75.45,146.33,639.63,339.19
2011-01-19,0,0,0,0,0,0,0,NaN,75.00,151.22,631.75,337.39


In [221]:
def calculate_leverage(trade, date, symbols):
    lev_abs_sum = 0
    lev_sum = 0
    for tick in symbols:
        share_cum_col = str(tick+'_cum')
        stock_price_col = str(tick+'_price')
        stock_position = trade.ix[date, share_cum_col] * trade.ix[date, stock_price_col]
        lev_abs_sum += np.abs(stock_position)
        lev_sum += stock_position
    cash = trade.ix[date, 'cash']
    leverage = lev_abs_sum / (lev_sum + cash)
    return leverage
    
for i in range(len(orders)):
    row = orders.ix[i]
    number_share = row['Shares']
    tick = row['Symbol']
    date = orders.index[i]
    
    if row['Order'] == 'SELL':
        buy_sell = -1.0
    elif row['Order'] == 'BUY':
        buy_sell = 1.0   
    else:
        print "Wrong value."
    
    trade_orig = trade.copy()
    trade.ix[date, tick] += (number_share * buy_sell)
    transaction_cost = 9.95
    trade.ix[date, 'transaction'] -= prices.ix[date, tick] * number_share * (buy_sell + 0.005) + transaction_cost
    # cumsum of stock shares
    trade[[str(tick+'_cum') for tick in symbols]] = trade[symbols].cumsum()
    # cumsum of transactions
    trade['transaction_cum'] = trade['transaction'].cumsum()
    # calculate cash in hand
    trade['cash'] = trade['transaction_cum'] + start_val
                 
    leverage = calculate_leverage(trade, date, symbols)
    if leverage > 2.0:
        trade = trade_orig
        print "forbiden"


#     print "Stock:", row['Symbol']
#     print "Order type:", row['Order']
#     print "Cost of order:", orderCost
#     print "Date order placed:", date, "\n"


forbiden


In [203]:
trade #-537059.25  303848.00

,order,leverage,transaction,AAPL,IBM,XOM,cash,AAPL_price,IBM_price,XOM_price,AAPL_cum,IBM_cum,XOM_cum
2011-01-10,1000000,0,-514052.375,1500.0,0.0,0.0,0,340.99,143.41,72.02,1500.0,0.0,0.0
2011-01-11,0,0,0.000,0.0,0.0,0.0,NaN,340.18,143.06,72.56,1500.0,0.0,0.0
2011-01-12,0,0,0.000,0.0,0.0,0.0,NaN,342.95,144.82,73.41,1500.0,0.0,0.0
2011-01-13,0,0,-67392.400,-1500.0,4000.0,0.0,NaN,344.20,144.55,73.54,0.0,4000.0,0.0
2011-01-14,0,0,0.000,0.0,0.0,0.0,NaN,346.99,145.70,74.62,0.0,4000.0,0.0
2011-01-15,0,0,0.000,0.0,0.0,0.0,NaN,346.99,145.70,74.62,0.0,4000.0,0.0
2011-01-16,0,0,0.000,0.0,0.0,0.0,NaN,346.99,145.70,74.62,0.0,4000.0,0.0
2011-01-17,0,0,0.000,0.0,0.0,0.0,NaN,346.99,145.70,74.62,0.0,4000.0,0.0
2011-01-18,0,0,0.000,0.0,0.0,0.0,NaN,339.19,146.33,75.45,0.0,4000.0,0.0
2011-01-19,0,0,0.000,0.0,0.0,0.0,NaN,337.39,151.22,75.00,0.0,4000.0,0.0


In [190]:
# cumsum of stock shares
trade[[str(tick+'_cum') for tick in symbols]] = trade[symbols].cumsum()
# cumsum of transactions
trade['transaction_cum'] = trade['transaction'].cumsum()
# calculate cash in hand
trade['cash'] = trade['transaction_cum'] + start_val
trade['stock_value'] = 0
for tick in symbols:
    print tick
    trade['stock_value'] += trade[str(tick+'_cum')] * trade[str(tick+'_price')] 
trade['portval'] = trade['cash'] + trade['stock_value']

AAPL
IBM
XOM


In [191]:
trade

,order,leverage,transaction,AAPL,IBM,XOM,cash,AAPL_price,IBM_price,XOM_price,AAPL_cum,IBM_cum,XOM_cum,transaction_cum,stock_value,portval
2011-01-10,1000000,0,-1542157.125,4500.0,0.0,0.0,-542157.125,340.99,143.41,72.02,4500.0,0.0,0.0,-1542157.125,1534455.0,992297.875
2011-01-11,0,0,0.000,0.0,0.0,0.0,-542157.125,340.18,143.06,72.56,4500.0,0.0,0.0,-1542157.125,1530810.0,988652.875
2011-01-12,0,0,0.000,0.0,0.0,0.0,-542157.125,342.95,144.82,73.41,4500.0,0.0,0.0,-1542157.125,1543275.0,1001117.875
2011-01-13,0,0,-134784.800,-3000.0,8000.0,0.0,-676941.925,344.20,144.55,73.54,1500.0,8000.0,0.0,-1676941.925,1672700.0,995758.075
2011-01-14,0,0,0.000,0.0,0.0,0.0,-676941.925,346.99,145.70,74.62,1500.0,8000.0,0.0,-1676941.925,1686085.0,1009143.075
2011-01-15,0,0,0.000,0.0,0.0,0.0,-676941.925,346.99,145.70,74.62,1500.0,8000.0,0.0,-1676941.925,1686085.0,1009143.075
2011-01-16,0,0,0.000,0.0,0.0,0.0,-676941.925,346.99,145.70,74.62,1500.0,8000.0,0.0,-1676941.925,1686085.0,1009143.075
2011-01-17,0,0,0.000,0.0,0.0,0.0,-676941.925,346.99,145.70,74.62,1500.0,8000.0,0.0,-1676941.925,1686085.0,1009143.075
2011-01-18,0,0,0.000,0.0,0.0,0.0,-676941.925,339.19,146.33,75.45,1500.0,8000.0,0.0,-1676941.925,1679425.0,1002483.075
2011-01-19,0,0,0.000,0.0,0.0,0.0,-676941.925,337.39,151.22,75.00,1500.0,8000.0,0.0,-1676941.925,1715845.0,1038903.075


In [212]:
def compute_portvals(orders_file="./orders/orders.csv", start_val=1000000):
    # Add orders to dataframe
    orders = pd.read_csv(orders_file, index_col=0, parse_dates=True, sep=',')
    orders = orders.sort_index()

    dates = pd.date_range(orders.index[0], orders.index[-1])
    dates = get_data(['SPY'], dates).index.get_values()
    print dates.shape
    symbols = orders['Symbol'].unique().tolist()

    # get price data
    prices = get_data(symbols, dates)[symbols]
    prices = pd.concat([prices, pd.DataFrame(index=dates)], axis=1)
    prices = prices.fillna(method='ffill')

    # create framework of trade table
    col = ['order', 'leverage', 'transaction'] + symbols
    trade = pd.DataFrame(columns=col + ['cash'], index=[dates])
    trade.ix[0, ['order']] = start_val
    trade[col] = trade[col].fillna(value=0)
    trade.ix[0, 'cash'] = 0
    trade = trade.merge(prices, how='left', left_index=True, right_index=True,
                        suffixes=('', '_price'))

    for i in range(len(orders)):
        row = orders.ix[i]
        number_share = row['Shares']
        tick = row['Symbol']
        date = orders.index[i]

        if row['Order'] == 'SELL':
            buy_sell = -1.0
        elif row['Order'] == 'BUY':
            buy_sell = 1.0
        else:
            print "Wrong value."

        trade.ix[date, tick] += (number_share * buy_sell)
        transaction_cost = 9.95
        market_impact = 0.005
        trade.ix[date, 'transaction'] -= prices.ix[date, tick] \
                                         * number_share \
                                         * (buy_sell + market_impact) \
                                         + transaction_cost

    # cumsum of stock shares
    trade[[str(tick+'_cum') for tick in symbols]] = trade[symbols].cumsum()
    # cumsum of transactions
    trade['transaction_cum'] = trade['transaction'].cumsum()
    # calculate cash in hand
    trade['cash'] = trade['transaction_cum'] + start_val
    # calculate the value of the current stock holding
    trade['stock_value'] = 0
    for tick in symbols:
        print tick
        trade['stock_value'] += trade[str(tick+'_cum')] * trade[str(tick+'_price')] 
    # add the cash and stock_value to get the portfolio value
    trade['portval'] = trade['cash'] + trade['stock_value']
    print trade['portval'].shape
    
    return pd.DataFrame(trade['portval'])

In [213]:
compute_portvals(orders_file="orders/orders-01-noleverage.csv", start_val=1000000)

(245,)
AAPL
IBM
XOM
GOOG
(245,)


,portval
2011-01-10,9.974326e+05
2011-01-11,9.962176e+05
2011-01-12,1.000373e+06
2011-01-13,9.967552e+05
2011-01-14,1.001355e+06
2011-01-18,1.003875e+06
2011-01-19,1.023435e+06
2011-01-20,1.023875e+06
2011-01-21,1.022715e+06
2011-01-24,1.038755e+06
